# Standard Layers

We will now see an overview of the enourmous diversity in deep learning layers. This survey is necessarily limited to standard layers and we begin *without* considering the key layers that enable deep learning of molecules and materials. Almost all the layers listed below came out of a model for a specific task and were not thought-up independently. That means that some of the layers are suited to specific tasks and often the nomenclature around that layer is targeted towards a specific kinds of data. The most common type is image data and we first begin with an overview of how image features are represented. An image is a rank 3 tensor with shape $(H, W, C)$ where $H$ is the height of the image, $W$ is the width, and $C$ is the number of channels (typically 3 -- red, green, blue). Since all training is in batches, the input features shape will be $(B, H, W, C)$. Often layers will discuss input as having a batch axis, some number of shape axes, and then finally a channel axis. The layers will then operate on perhaps only the channels or only the shape dimensions. The layers are all quite flexible, so this is not a limitation in practice, but it's important to know when reading about layer types. Often the documentation or literature will mention *channels* or *batch number* and this is typically the first and last axes of a tensor, respectively. 

```{note}
Everything and nothing is batched in deep learning. Practically, data is always batched. Even if your data is not batched, the first axis input to a neural network is of unspecified dimension and called the batch axis. Many frameworks make this implicit, meaning if you say the output from one layer is shape $(4,5)$, it will be $(B, 4, 5)$ when you actually inspect data. So, all data is batched but often the math, frameworks, and documentation make it seem as if there is no batch axis. 
```

## Hyperparameters

We saw from the FC/Dense layer that we have to choose if we use bias, the activation, and the output shape. As we learn about more complex layers, there will be more choices. These choices begin to accumulate and in a neural network you may have billions of possible combinations of them. These chocies about shape, activation, initialization, and other layer arguments are called **hyperparameters**. They are parameters in the sense that they can be tuned, but they are not trained on our data so we call them hyperparameters to distinguish them from the "regular" parameters like value of weights and biases in the layers. The name is inherited from Bayesian statistics. 

Choosing these hyperparameters is difficult and we typically rely on the body of existing literature to understand ranges of reasonable parameters. In deep learning, we usually are in a regime of hyperparameters which yield many trainable parameters (deep networks) and thus our models can represent any function. Our models are expressive. However, optimizing hyperparameters makes training faster and/or require less data. For example, papers have shown that carefully choosing the initial value of weights can be more effective than complex architecture {cite}`glorot2010understanding`. Another example found that convolutions, which are thought to be the most important layer for image recognition, are not necessary if hyperparameters are chosen correctly for dense neural networks{cite}`noconv`. This is now changing, with options for tuning hyperparameters, but the current state-of-the art is to take hyperparameters from previous work as a starting guess and change a little if you believe it is needed. 

### Validation

The number of hyperparameters is high-enough that overfitting can actually occur by choosing hyperparameters that minimize error on the test set. This is surprising because we don't explicitly train hyperparametes. Nevertheless, you will find in your own work that if you use the test data extensively in hyperparameter tuning and for assessing overfitting of the regular training parameters, your performance will be overfit to the testing data. To combat this, we split our data three ways in deep learning:

1. Training data: used for trainable parameters. 
2. Validation data: used to choose hyperparameters or measure overfitting of training data
3. Test data: data not used for anything except final reported error 

To clean-up our nomenclature here, we use the word **genaralization error** to refer to performance on a hypothetical infinite stream of unseen data. So regardless of if you split three-ways or use other approaches, generalization error means error on unseen data. 

```{margin}
You can replace this three-way split with cross-validation methods from previously, but remember that those require training $k$-times. Thus you rarely see k-fold corss-validation and more rarely see leave-one-out or Jacknife because it is expensive to train models.
```

### Tuning

So how do you tune hyperparameters? The main answer is by-hand, but this is an active area of research. Hyperparameters are continuous (e.g., regularization strength), categorical (e.g., which activation), and discrete variables (e.g., number of layers). One category of ways to tune hyperparameters is a topic called meta-learning{cite}`finn2017model`, which aims to learn hyperparameters by looking at multiple related datasets. Another area is auto-machine learning (auto-ML){cite}`45826`, where optimization strategies that do not require derivatives can tune hyperparameters. An important category of optimization related to hyperparameter tuning is **mult-armed bandit** optimization where we explicitly treat the fact that we have a finite amount of computational resources for tuning hyperparameters{cite}`hyperband`. Specific implementations of these methods can be found in many ML frameworks now, for example [Keras-Tuner for Keras](https://keras-team.github.io/keras-tuner/) or [Ray Tune](https://docs.ray.io/en/latest/tune/index.html) for PyTorch. 

## Common Layers

Now that we have some understanding of hyperparameters and their role, let's now survey the common types of layers.

### Convolutions

You can find a more thorough overview of [convolutions here](http://d2l.ai/chapter_convolutional-neural-networks/why-conv.html). Convolutions are the most commonly used input layer when dealing with images or other data defined on a regular grid. In chemistry, you'll see convolutions on protein or DNA sequences, on 2D imaging data, and occasionally on 3D spatial data like average density from a molecular simulation. What makes a convolution different from an dense layer is that the number of trainable weights is more flexible than input grid shape $\times$ output shape. Since the trainable parameters doesn't depend on the input grid shape, you can in principle work with any size images or arbitrary length DNA sequences. This is impossible with dense layers, which must have all input be the same shape.

In a convolution, you specify a **kernel shape** that defines the size of trainable parameters. The kernel shape defines a window over your input data in which a dense neural network is applied. The rank of the kernal shape is the rank of your grid + 1, where the extra axis accounts for channels. For example, for images you might define a kernel shape of $12\times12$. The kernel shape will become $12\times12\times{}C$, where $C$ is the number of channels. When referring to a convolution as 1D, 2D, or 3D, we're referring to the grid of the input data and thus the kernel shape. A 2D convolution actually works on rank 4 tensors, the extra 2 axes accounting for batch and channels.

Practically, convolutions are always grouped in in parallel. You have a set of $F$ kernels, where $F$ is called the number of **filters**. Each of these filters is completely independent and in fact if you examine what they learn, some filters will learn to identify squares and some might learn to identify color or others will learn textures. Filters is a term left-over from image processing, which is the field where convolutions were first explored. Combing all of these together, a 2D convolution will have an input shape of $(B, H, W, C)$ and an output of $(B, \approx H, \approx W, F)$, where $F$ is the number of filters chosen, and the $\approx$ accounts for the fact that when you slide your kernel window over the input data, you'll lose some values on the edge. This can either be treated by padding, so your input height and width match output height and width, or your dimensionality is reduced by a small amount (e.g., going from $128\times128$ to $125\times125$). A 1D convolution will have input shape $(B, L, C)$ and output shape $(B, \approx L, F)$. As a practical example, consider a convolution on DNA. $L$ is length of the sequence. $C$, your channels, will be one-hot indicators for the base (T, C, A, G).

One of the important properties we'll begin to discuss is **invariances** and **equivariances**. An invariance means the output from a neural network (or a general function) is insensitive to changes in input. For example, a translational invariance means that the output does not change if the input is translated. Convolutions and pooling should be chosen when you want to have **translation invariance**. For example, if you are identifying if a cat exists in an image, you want your network to give the same answer even if the cat is translated in the image to different regions. However, just because you use a convolution layer does not make a neural network automatically translationally invariant. You must include other layers to achieve this.  Convolutions are actually translationally equivariant -- if you translate all pixels in your input, the output will also be translated. People usually do not distinguish between equivariance and invariance. If you are trying to identify *where* a cat is located in an image you would still use convolutions but you want your neural network to be translationally equivariant, meaning your guess about where the cat is located is sensitive to where the cat is located in the input pixels. The reason convolutions have this property is that the trainable parameters, the kernel, are location independent. You use the same kernel on every region of the input. 

### Pooling

Convolutions are commonly paired with pooling layers. If your goal is to produce a single number (regression) or class (classification) from an input image or sequence, you need to reduce the rank to 0, a scalar. After a convolution, you could use a reduction like average or maximum. It has been shown empirically that reducing the number of elements of your features more gradually is better. One way is through **pooling**. Pooling is similar to convolutions, in that you define a kernel shape (called window shape), but pooling has no trainable parameters. Instead, you run a window across your input grid and compute a reduction. Commonly an average or maximum is computed. If your pool window is a $2\times2$ on an input of $(B, H, W, F)$, then your output will be $(B, H / 2, W / 2, F)$. In convolutional neural networks, often multiple **blocks** of convolutions and poolings are combined. For example, you might use three rounds of convolutions and pooling to take an image from $32 \times 32$ down to a $4 \times 4$. Pooling is translationally equivariant like convolutions. Read more about [pooling here](http://d2l.ai/chapter_convolutional-neural-networks/pooling.html)



### Embedding

Another important type of input layers are **embeddings**. Embeddings convert integers into vectors. They are typically used to convert characters or words into numerical vectors. The characters or words are first converted into **tokens** separately as a pre-processing step and then the input to the embedding layer is the indicies of the token. The indices are integer vales that index into a dictionary of all possible tokens. It sounds more complex than it is. For example, we might tokenize characters in the alphabet. There are 26 tokens (letters) in the alphabet (dictionary of tokens) and we could convert the word "hello" into the indices $[7, 4, 4, 11, 11, 14]$, where 7 means the 7th letter of the alphabet. 

After converting into indices, an embedding layer converts these indices into dense vectors of a chosen dimension. The rationale behind embeddings is to go from a large discrete space (e.g., all words in the English language) into a much smaller space of real numbers (e.g., vectors of size 5). You might use embeddings for converting monomers in a polymer into dense vectors or atom identities in a molecule or DNA bases.

### Example

At this point, we have enough common layers to try to build a neural network. 

### Regularization

#### Kernel

#### Activity

#### Batch Normalization

#### Dropout


### Attention

## Initialization

## Residues

## Grouping Layers into a Block

## Cited References

```{bibliography} references.bib
```